In [1]:
import os
import glob
import numpy as np
import pandas as pd
import ast 
import statistics
from datetime import datetime 

def get_prefix_from_path(file_path):
    if not file_path:
        return 'default'

    base_name = os.path.splitext(os.path.basename(file_path))[0]
    parts = base_name.split('_')
    
    if len(parts) >= 2:
        return '_'.join(parts[:2])
    else:
        return base_name

def quality(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,input_file_path6,prefix):

    df=pd.read_csv('quality.csv')
    df=pd.DataFrame(df)

    data_add = pd.read_csv(input_file_path1)['指标值'] if input_file_path1 else np.zeros(300, dtype=int)
    data_remove = pd.read_csv(input_file_path2)['指标值'] if input_file_path2 else np.zeros(300, dtype=int)
    data_sum = pd.read_csv(input_file_path3)['指标值'] if input_file_path3 else np.zeros(300, dtype=int)
    data_review = pd.read_csv(input_file_path4)['指标值'] if input_file_path4 else np.zeros(300, dtype=int)
    data_request = pd.read_csv(input_file_path5)['指标值'] if input_file_path5 else np.zeros(300, dtype=int)
    data_accepted = pd.read_csv(input_file_path6)['指标值'] if input_file_path6 else np.zeros(300, dtype=int)


    sum_request=sum(data_request)
    sum_review=sum(data_review)
    if(sum_request!=0):
        avg1=sum_review/sum_request
    else :
        avg1=0
    s_request=[]
    for i in range(len(data_request)) :
        if data_request[i]!=0:
            s_request.append(data_accepted[i]/data_request[i])
        else:
            s_request.append(0)
    avg2=np.mean(s_request)

    s_add=[]
    s_remove=[]
    for i in range(len(data_sum)):
        if data_sum[i]!=0:
            s_add.append(data_add[i]/np.abs(data_sum[i]))
            s_remove.append(data_remove[i]/np.abs(data_sum[i]))
        else :
            s_add.append(0)
            s_remove.append(0)
    avg3=np.mean(s_add)
    avg4=np.mean(s_remove)
    avg5=avg3-avg4
    final=avg1/4+avg2/4+abs(avg5/2)
    # 获取仓库名
    row_exists=(df['指标名称']==prefix).any()

    if row_exists:
        df.loc[df['指标名称'] == prefix, '指标值']=final*10

    # 保存更改回 CSV 文件
    df.to_csv('quality.csv', index=False)

def get_prefix_and_rest(filename):

    base_name = os.path.basename(filename)
    # 分离文件名和扩展名
    name_part, ext_part = os.path.splitext(base_name) 
    # 按下划线分割文件名部分，并确保至少有两个'_'
    parts = name_part.split('_')

    if len(parts) > 1:
        # 获取前两个'_'之间的部分作为前缀
        prefix = '_'.join(parts[:2])
        # 剩余部分加上原始扩展名作为后缀
        rest = '_'.join(parts[2:]) + ext_part if len(parts) > 2 else ext_part
    else:
        # 如果没有足够的'_'，则返回整个文件名作为前缀，空字符串作为后缀
        prefix = name_part
        rest = ext_part
    
    return prefix, rest

def process_files_with_same_prefix(base_dir):
    # 定义需要查找的文件后缀
    file_suffixes = [
        'code_change_lines_add.csv','code_change_lines_remove.csv','code_change_lines_sum.csv',
        'change_requests_reviews.csv','change_requests.csv','change_requests_accepted.csv',
        'active_dates_and_times.csv'
    ]
    
    # 获取所有csv文件
    all_files = glob.glob(os.path.join(base_dir, '*.csv'))
    # 创建一个字典来存储每个前缀对应的文件路径列表
    files_dict = {}
    
    for file in all_files:
        
        # 获取仓库和后缀的部分作为前缀
        prefix, suffix = get_prefix_and_rest(os.path.basename(file))
        
        key = next((s for s in file_suffixes if s==suffix), None)
        if key:  # prefix是仓库名，key是文件后缀匹配，file是路径
            if prefix not in files_dict:
                files_dict[prefix] = {}
            files_dict[prefix][key] = file
    # 遍历字典并调用activity函数
    for prefix, paths in files_dict.items():
        # 调用 activity 函数，传递对应前缀的所有文件路径
        input_file_path1=paths.get('code_change_lines_add.csv')
        input_file_path2=paths.get('code_change_lines_remove.csv')
        input_file_path3=paths.get('code_change_lines_sum.csv')
        input_file_path4=paths.get('change_requests_reviews.csv')
        input_file_path5=paths.get('change_requests.csv')
        input_file_path6=paths.get('change_requests_accepted.csv')

        quality(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,input_file_path6,prefix)

# 假设 base_dir 是你的文件夹路径
base_dir='./最终数据处理'
process_files_with_same_prefix(base_dir)